In [15]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch

In [16]:
# Verificar la disponibilidad de CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [17]:
# Cargar datos de Bitcoin
data = pd.read_csv('ETHUSDT1days6monthn.csv')  # Asegúrate de tener tu propio archivo CSV de datos de Bitcoin
data.head()

# Preprocesar datos
# ... (proceso de preprocesamiento adecuado para tu conjunto de datos)

,Unnamed: 0,Date,Open,High,Low,Close,Volume,EMA4,EMA9,EMA18,RSI,STOCHRSIK,STOCHRSID,MACDs,MACDh,MACD,BBUPPER,BBMIDDLE,BBLOWER,ADX
0,2021-08-21 21:00:00,3223.36,3272.00,3126.42,3238.00,406479.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-22 21:00:00,3238.00,3377.89,3230.02,3321.28,602103.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-23 21:00:00,3321.28,3357.85,3147.31,3170.62,540480.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-24 21:00:00,3170.63,3248.00,3080.00,3226.87,483697.69,3239.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-25 21:00:00,3226.88,3250.00,3053.30,3093.20,442298.47,3180.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Supongamos que tenemos una columna 'precio' en los datos
# Convierte la columna de precios a un arreglo numpy
prices = data['Close'].values.astype(float)

# Normalización de datos (escala entre 0 y 1)
max_price = np.max(prices)
min_price = np.min(prices)
normalized_prices = (prices - min_price) / (max_price - min_price)

# Función para crear secuencias
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length:i+seq_length+1]  # Predicción siguiente punto
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Crear secuencias
seq_length = 10
X, y = create_sequences(normalized_prices, seq_length)

# Dividir en conjunto de entrenamiento y prueba
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


X_train = torch.tensor(X_train, dtype=torch.float32).cuda()
y_train = torch.tensor(y_train, dtype=torch.float32).cuda()
X_test = torch.tensor(X_test, dtype=torch.float32).cuda()
y_test = torch.tensor(y_test, dtype=torch.float32).cuda()

In [20]:
prices

array([ 406479.08,  602103.45,  540480.08,  483697.69,  442298.47,
        451027.81,  262360.35,  335806.78,  625347.28,  863071.13,
        845759.96,  525165.8 ,  655903.76,  364478.12,  365759.86,
        397768.58, 1203536.  ,  780611.77,  628553.88,  633026.27,
        413524.18,  408858.96,  647855.72,  373634.9 ,  428507.53,
        467985.49,  398888.37,  315162.76,  312948.  ,  917939.58,
        971519.89,  677457.65,  459293.89,  932090.89,  491974.02,
        703594.48,  510130.73,  511325.46,  411305.09,  506896.76,
        648714.62,  422551.18,  360586.89,  485204.64,  393900.88,
        618265.54,  486102.36,  403597.14,  361906.75,  580223.63,
        493521.88,  409377.19,  393463.12,  481894.15,  504321.73,
        373727.85,  368898.98,  401819.97,  341268.37,  537383.1 ,
        798566.56,  463222.54,  326576.96,  350115.84,  347438.65,
        414498.16,  704604.25,  626378.51,  559231.08,  340223.07,
        412856.29,  417620.71,  468184.79,  448658.35,  368218

In [19]:
data.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,EMA4,EMA9,EMA18,RSI,STOCHRSIK,STOCHRSID,MACDs,MACDh,MACD,BBUPPER,BBMIDDLE,BBLOWER,ADX
0,2021-08-21 21:00:00,3223.36,3272.00,3126.42,3238.00,406479.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-22 21:00:00,3238.00,3377.89,3230.02,3321.28,602103.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-23 21:00:00,3321.28,3357.85,3147.31,3170.62,540480.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-24 21:00:00,3170.63,3248.00,3080.00,3226.87,483697.69,3239.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-25 21:00:00,3226.88,3250.00,3053.30,3093.20,442298.47,3180.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).cuda(),
                            torch.zeros(1,1,self.hidden_layer_size).cuda())

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Definir parámetros
input_size = 1
hidden_layer_size = 100
output_size = 1

# Crear modelo y enviar a GPU
model = LSTM(input_size, hidden_layer_size, output_size).cuda()

# Definir función de pérdida y optimizador
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
epochs = 1000

for i in range(epochs):
    for seq, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).cuda(),
                        torch.zeros(1, 1, model.hidden_layer_size).cuda())

        y_pred = model(seq)

        single_loss = criterion(y_pred, labels)
        single_loss.backward()
        optimizer.step()

        # Opcional: Imprimir la pérdida
        print(f'Epoch {i} Loss: {single_loss.item()}')

# Evaluar el modelo en el conjunto de prueba
model.eval()
with torch.no_grad():
    test_predictions = []
    for seq in X_test:
        seq = seq.view(-1, 1)
        test_predictions.append(model(seq).item())

# Desnormalizar las predicciones
test_predictions = np.array(test_predictions) * (max_price - min_price) + min_price

Epoch 0 Loss: 0.3035348057746887
Epoch 0 Loss: 0.04946935921907425
Epoch 0 Loss: 0.10823991894721985
Epoch 0 Loss: 0.0009513571276329458
Epoch 0 Loss: 0.0003439132124185562
Epoch 0 Loss: 0.001364471740089357
Epoch 0 Loss: 0.6285504698753357
Epoch 0 Loss: 0.13030189275741577
Epoch 0 Loss: 0.03818628564476967
Epoch 0 Loss: 0.03281855955719948
Epoch 0 Loss: 0.0023045213893055916
Epoch 0 Loss: 0.004258532077074051
Epoch 0 Loss: 0.022888682782649994
Epoch 0 Loss: 0.017021065577864647
Epoch 0 Loss: 0.007421995513141155
Epoch 0 Loss: 0.0030962685123085976
Epoch 0 Loss: 0.016337189823389053
Epoch 0 Loss: 0.043076224625110626
Epoch 0 Loss: 0.041008442640304565
Epoch 0 Loss: 0.1474032998085022
Epoch 0 Loss: 0.17426683008670807
Epoch 0 Loss: 0.013136139139533043
Epoch 0 Loss: 0.011079099960625172
Epoch 0 Loss: 0.11841998994350433
Epoch 0 Loss: 0.009966743178665638
Epoch 0 Loss: 0.009524512104690075
Epoch 0 Loss: 0.010374042205512524
Epoch 0 Loss: 0.01094746682792902
Epoch 0 Loss: 0.04038424789905